# Semantic Search
Semantic search is [defined as "search with meaning."](https://en.wikipedia.org/wiki/Semantic_search) It is key for effective knowledge retrieval.

As opposed to traditional, lexical, search which finds matches based on keywords, semantic search seeks to improve search quality and accuracy by understanding search intent and pulling results that match the user’s contextual meaning.  Semantic search is often used in reference to text embedding and vector similarity search, but this is just one implementation aspect of it. Knowledge graph and symbolic query logic can also play a critical role in making semantic search a reality. 

If all you care about is analyzing a set of documents on a file system, then sure, vector indexing and search may be sufficient.  However, once you need to retrieve and make inferences about people, places, and things connected to those documents, Knowledge graph becomes key. 

If documents are the entities of interest, for example: "find all documents that talk about pharma related things," then text embeddings with vector similarity search suffices.  But, what if we want second or third-order entities related to the documents?  For example, "find investors who are most focused on pharma related strategies," how would we efficiently search for them at scale in an enterprise setting?

This is what we demonstrate below.  We will also show how you can use graph relationships and Graph Data Science algorithms to further improve search results, especially in common scenarios where the presence of text data is inconsistent or sparse.

## Connect to Neo4j

In [ ]:
# username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match
NEO4J_URI = 'neo4j+s://xxxxx.databases.neo4j.io'
NEO4J_PASSWORD = 'password'

In [ ]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')

## Neo4j Vector Index
We will need to create a vector index for similarity search on Document nodes. Neo4j offers a vector index that enables Approximate Nearest Neighbor Search (ANN). Let's creat an index.

In [ ]:
gds.run_cypher("CALL db.index.vector.createNodeIndex('document-embeddings', 'Document', 'textEmbedding', 768, 'cosine')")

You can see that vector index has been created using `SHOW INDEXES`

In [ ]:
gds.run_cypher(''' 
SHOW INDEXES YIELD name, type, labelsOrTypes, properties, options
WHERE type = "VECTOR"
''')

## Deep Semantic Search with Knowledge Graph
Now that we have an index.  Let’s use it in action.
In this case, we will Answer The Question - "What Investors are most focused in pharma, medicine, and healthcare?" 
Remember we do not have documents on investment managers, just companies, and there can be multiple documents for each company. 

In [ ]:
# Semantic query, lets use these key words to search
semantic_query = ['pharma, medicine, healthcare']

In [ ]:
from vertexai.language_models import TextEmbeddingModel

# Create a query vector by embedding the query using Vertex AI text embedding
EMBEDDING_MODEL = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [ ]:
# Get query vector
query_vector = EMBEDDING_MODEL.get_embeddings(semantic_query)[0].values
query_vector[:10]

Now let's use that query vector to search for companies.  Remember, companies have multiple documents so we will need to use a graph traversal on top of a document lookup to find which companies are most similar.

In [ ]:
# Search for similar companies
res_df = gds.run_cypher("""
CALL db.index.vector.queryNodes('document-embeddings', 1000, $queryVector)
YIELD node AS similarDocuments, score
MATCH (similarDocuments)<-[:HAS]-(c:Company)
RETURN c.companyName as companyName, avg(score) AS score
ORDER BY score DESC LIMIT 100
""", params =  {'queryVector': query_vector})
res_df

You may recognize some of these companies, and if not a quick google search will con firm that their business is involved in healthcare and pharma, so this seems to be working.

Now let's take this one step further and find investment managers who are most heavily focused in pharma.  This will involve a bit more Cypher for a 2-hop traversal. 

In [ ]:
# Search for managers with significiant investments in area
res_df = gds.run_cypher("""
CALL db.index.vector.queryNodes('document-embeddings', 1000, $queryVector)
YIELD node AS similarDocuments, score
MATCH (similarDocuments)<-[:HAS]-(c:Company)
WITH c, avg(score) AS score ORDER BY score LIMIT 100
MATCH (c)<-[r:OWNS]-(m:Manager)
WITH m, r.value as value, score*r.value as weightedScore
WITH m.managerName AS managerName, sum(weightedScore) AS aggScore, sum(value) AS aggValue
RETURN managerName, aggScore/aggValue AS score ORDER BY score DESC LIMIT 1000

""", params =  {'queryVector': query_vector})
res_df

And we can see that our top result is a specialized investment trust in Bio-Technology
[INTERNATIONAL BIOTECHNOLOGY TRUST PLC](https://ibtplc.com/)

## Expanding Available Data for Knowledge Retrieval
Not every element in your data will have rich text data, and further, much like we only have 10K documents for some companies, your use cases may also have incomplete, unevenly distributed text data. 

We can check our top result investment manager to this. 

In [ ]:
gds.run_cypher('''
MATCH (m:Manager {managerName: $managerName})-[:OWNS]->(c:Company)-[:HAS]->(d:Document)
WITH m, count(DISTINCT c) AS ownedCompaniesWithDocs
MATCH (m:Manager {managerName: $managerName})-[:OWNS]->(c:Company)
RETURN m.managerName AS managerName, ownedCompaniesWithDocs, count(DISTINCT c) AS totalOwnedCompanies
''', params =  {'managerName':'INTERNATIONAL BIOTECHNOLOGY TRUST PLC'})

This manager has significantly more other companies they own without documents.  We can use Graph Data Science Node Similarity to find the managers that have the most overlap to this one which should give us other Biotech companies that we missed due to sparse text data.

In [ ]:
g, _  = gds.graph.project('proj', ['Company', 'Manager'], {'OWNS':{'properties':['value']}})

In [ ]:
gds.nodeSimilarity.write(g, writeRelationshipType='SIMILAR', writeProperty='score', relationshipWeightProperty='value')

In [ ]:
g.drop()

And now we can pull back other relevant results

In [ ]:
gds.run_cypher('''
MATCH (m0:Manager {managerName: $managerName})-[r:SIMILAR]->(m:Manager)
RETURN m.managerName AS managerName, r.score AS score
ORDER BY score DESC LIMIT 10
''', params =  {'managerName':'INTERNATIONAL BIOTECHNOLOGY TRUST PLC'})

## Clean Up

In [ ]:
gds.run_cypher('MATCH (M:Manager)-[s:SIMILAR]->() DELETE s')